# Рынок вакансий аналитиков данных на LinkedIn

<div style="border: solid seagreen 1px; padding: 10px">
    
**Задача** — визуализировать информацию о рынке вакансий для аналитиков данных начального уровня (data analyst и BI analyst) в Европе. 

**Источник информации**: база в формате csv, содержащая код HTML-страниц с описанием вакансий на LinkedIn по состоянию на 23 мая 2023 года.
    
В рамках задачи будут выполнены следующие шаги:
- [загрузка и обзор данных](#overview),
- [предобработка данных](#data_processing),
- [выгрузка данных](#extraction),
- [построение дашборда](#dash).

In [1]:
# импортируем библиотеки
import pandas as pd
import requests
from urllib.parse import urlencode
import json
from IPython.display import HTML
from bs4 import BeautifulSoup
import re
import numpy as np
import datetime as dt

# настраиваем отображение колонок датафрейма при выводе данных
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

<a id='overview'></a>
## Загрузка и обзор данных

In [2]:
# загружаем данные

try:
    df = pd.read_csv('/Users/mrmrzpn/Desktop/Yandex Praktikum/\
7. Мастерская/LinkedIn/masterskaya_parsing_LinkedIn_2023_05_23.csv',
                    usecols=['html'])
except:
    # используем api 
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
    # ссылка на файл в облаке
    public_key = 'https://disk.yandex.ru/d/Rlo2KdJRve6fkw'
    
    # получаем url для запроса на скачивание файла
    url = base_url + urlencode(dict(public_key=public_key))
    # отправляем запрос на скачивание файла
    response = requests.get(url).text
    # вытаскиваем из ответа прямую ссылку для скачивания файла
    download_url = json.loads(response)['href']
    
    # загружаем файл в датасет
    df = pd.read_csv(download_url, index_col=0)

In [3]:
# смотрим информацию о датасете

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   html    998 non-null    object
dtypes: object(1)
memory usage: 7.9+ KB


In [4]:
# смотрим для примера, как выглядит страница вакансии

HTML(df['html'][0])

**Наблюдения:** В датасете 998 вакансий, пропусков нет.

<a id='data_processing'></a>
## Предобработка данных

### Проверка на дубликаты - 1

In [5]:
# проверим, есть ли полные дубликаты

sum(df.duplicated())

98

В датасете 98 полных дубликатов, удалим их:

In [6]:
df = df.drop_duplicates(ignore_index=True)

### Парсинг

Нужно собрать следующую информацию:
- наименование вакансии
- наименование компании 
- город
- страна
- уровень кандидата (junior, middle, senior)
- тип занятости (удаленно/офис/гибридный график)
- дата публикации вакансии
- количество кандидатов на вакансию
- требуемые хард скилы
- сфера деятельности компании
- размер компании (количество сотрудников)

Требуемые хард скиллы возьмем из описания вакансии в следующем шаге.

In [7]:
# объявим функцию для парсинга данных

def get_data(html):
    soup = BeautifulSoup(html, 'lxml')
    
    # парсим данные, и если находим нужный признак, убираем лишние пробелы и перенос строки
    def parsing(tag_str, class_str):
        result = soup.find(tag_str, attrs={'class': class_str})
        if result:
            result = result.text.strip()
        return result
    
    # название вакансии
    job_title = parsing('h2', 't-24 t-bold jobs-unified-top-card__job-title')
    # название компании
    company = parsing('span', 'jobs-unified-top-card__company-name')
    # город и страна
    location = parsing('span', 'jobs-unified-top-card__bullet')
    # разделим город и страну
    if location:
        location = location.split(', ')
        city = location[0]
        country = location[-1]
    else:
        city = np.nan
        country = np.nan
    # ожидаемый уровень кандидатов
    level = parsing('li', 'jobs-unified-top-card__job-insight')
    if level:
        level = level.split('·')
        level = level[-1].strip()
    # тип занятости
    workplace = parsing('span', 'jobs-unified-top-card__workplace-type')
    # дата публикации
    posted_date = parsing('span', 'jobs-unified-top-card__posted-date')
    # число откликов
    applicants = parsing('span', 'jobs-unified-top-card__applicant-count')
    # описание вакансии
    description = parsing('article', 'jobs-description__container m4')
    # если в описании вакансии только заголовок, возвращаем nan
    if description == 'About the job':
        description = np.nan
    # индустрия и размер компании
    industry_size = parsing('div', 't-14 mt5')
    # разделим индустрию и размер компании
    if industry_size:
        industry_size = re.split('\n +', industry_size)
        industry = industry_size[0]
        try:
            size = industry_size[2]
        except:
            size = np.nan
    else:
        industry = np.nan
        size = np.nan
    
    return [job_title, company, city, country, level, workplace, posted_date, applicants, description,
            industry, size]
    

In [8]:
# создадим датафрейм по итогам парсинга

data = []
columns = ['job_title', 'company', 'city', 'country', 'level', 'workplace', 'posted_date',
           'applicants', 'description', 'industry', 'size']

for i in range(len(df)):
    data.append(get_data(df.loc[i,'html']))

parsed_df = pd.DataFrame(data, columns=columns)

In [9]:
# смотрим информацию о датафрейме

parsed_df.info()
parsed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job_title    900 non-null    object
 1   company      900 non-null    object
 2   city         900 non-null    object
 3   country      900 non-null    object
 4   level        883 non-null    object
 5   workplace    832 non-null    object
 6   posted_date  900 non-null    object
 7   applicants   740 non-null    object
 8   description  899 non-null    object
 9   industry     866 non-null    object
 10  size         866 non-null    object
dtypes: object(11)
memory usage: 77.5+ KB


job_title                                   company  \
0                     Data Analyst  PharmiWeb.Jobs: Global Life Science Jobs   
1         Data Analyst - Logistics                      Resolute Recruitment   
2         Data Analyst - Logistics                      Resolute Recruitment   
3  Data Analyst (Space & Planning)                       Mole Valley Farmers   
4                     Data Analyst                                   FORFIRM   

           city         country        level workplace  posted_date  \
0         Basel     Switzerland  Entry level   On-site   1 week ago   
1      Coventry  United Kingdom         None   On-site   1 week ago   
2      Coventry  United Kingdom         None   On-site   1 week ago   
3  South Molton  United Kingdom         None   On-site   1 week ago   
4        Lugano     Switzerland         None   On-site  2 weeks ago   

      applicants  \
0  47 applicants   
1           None   
2           None   
3           None   
4           None   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

**Наблюдения:** Не по всем вакансиям предоставлены полные данные: в некоторых случаях отсутствует информация о типе занятости, уровне кандидатов, количестве откликов на вакансию, индустрии и размере компании. В одном случае нет подробного описания вакансии с требованиями к кандидатам.  
Пропуски удалять не будем, чтобы аналитика по числу вакансий в разрезе компаний, стран, индустрий была полной.

### Расчет даты публикации

Дата публикации представлена в текстовом формате, в виде отрезка времени с момента публикации до 23 мая 2023 года. Изменим формат представления даты.

In [10]:
# посмотрим, какие отрезки времени есть в датасете

parsed_df['posted_date'].unique()

array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

In [11]:
# для перевода отрезка времени в дни составим словарь замен

days_number = {'week':7, 'weeks':7, 'day':1, 'days':1, 'hours':0, 'minutes':0}

In [12]:
# сохраним расчетную дату публикации в датасете

parsed_df['date'] = parsed_df['posted_date'].apply(lambda x: (dt.date(2023, 5, 23) - 
                                                              dt.timedelta(days = int(x.split(' ')[0]) * 
                                                                           days_number[x.split(' ')[1]])
                                                             )).astype('datetime64[D]')

### Проверка на релевантность

Проверим вакансии на соответствие требованиям: ищем аналитиков данных и BI аналитиков начального уровня в Европе.

#### Название вакансии

In [13]:
# посмотрим на названия вакансий

sorted(parsed_df['job_title'].unique())

['(ESG) Data Analyst (w/m/d)',
 '(Junior) Application Specialist eCommerce (m/f/d)',
 '(Junior) Business Analyst (f/m/d)',
 '(Junior) Business Analyst (w/m/d)',
 '(Junior) Consumer Business Intelligence Analyst (m/w/d)',
 '(Junior) Data Scientist',
 '(Junior) Project Analyst (m/f/d)',
 '(Junior) Project Coordinator',
 '(Junior) Systems Business Analyst (f/m/d)',
 '(Senior) Consultant Transaction Analytics (f/m/d)',
 '360 Data Product Owner',
 'AFC Data Analyst',
 'ALTERNANCE - Assistance Data Analyst et Reporting RH F/H',
 'ANALISTA (H/M) FINANCIERO / FP&A REPORTING',
 'ANALISTA DE PROYECTO DIGITALIZACIÓN',
 'ANALYSTE DES DONNEES (H/F)',
 'APM Software Development & Operations',
 'Accountant',
 'Aftersales Reporting Specialist',
 'Alternance - Data Analyst - Data ETL & Visualisation (h/f/d)',
 'Analista Dati di Geodesia - CATEGORIA PROTETTA',
 'Analista Ecommerce, Web y App',
 'Analista Funzionale IT (Junior)',
 'Analista de Software',
 'Analista de datos',
 'Analista de datos BI',
 'A

Актуальные вакансии должны включать слово "аналитик" и "Business Intelligence (BI)":

In [14]:
# составим список слов, которые должны обязательно встречаться в подходящих вакансиях

analyst = ['anal', 'dba', 'data research', 'intelligence', '\bbi\b', '\bbi-', 'visualization',
           'visualisation', 'data consultant', 'processing']

In [15]:
# оставим в датасете только аналитиков

parsed_df = (parsed_df.loc[
    parsed_df['job_title']
    .apply(lambda x: (sum([i in x.lower() for i in analyst]) != 0))]
            ).reset_index(drop=True)

In [16]:
# посмотрим на оставшиеся названия вакансий, чтобы исключить неподходящие

sorted(parsed_df['job_title'].unique())

['(ESG) Data Analyst (w/m/d)',
 '(Junior) Business Analyst (f/m/d)',
 '(Junior) Business Analyst (w/m/d)',
 '(Junior) Consumer Business Intelligence Analyst (m/w/d)',
 '(Junior) Project Analyst (m/f/d)',
 '(Junior) Systems Business Analyst (f/m/d)',
 '(Senior) Consultant Transaction Analytics (f/m/d)',
 'AFC Data Analyst',
 'ALTERNANCE - Assistance Data Analyst et Reporting RH F/H',
 'ANALISTA (H/M) FINANCIERO / FP&A REPORTING',
 'ANALISTA DE PROYECTO DIGITALIZACIÓN',
 'ANALYSTE DES DONNEES (H/F)',
 'Alternance - Data Analyst - Data ETL & Visualisation (h/f/d)',
 'Analista Dati di Geodesia - CATEGORIA PROTETTA',
 'Analista Ecommerce, Web y App',
 'Analista Funzionale IT (Junior)',
 'Analista de Software',
 'Analista de datos',
 'Analista de datos BI',
 'Analista de datos dept de elaborados (Guissona)',
 'Analista funzionale',
 'Analista superior de datos',
 'Analityk',
 'Analityk danych',
 'Analityk danych internetowych (wszystko.pl)',
 'Analyst',
 'Analyst (Strategy, Policy and Innova

Исключим из вакансий аналитиков-разработчиков, аналитиков бизнес-процессов, лабораторных аналитиков, дата-саентистов, инженеров данных, финансовых аналитиков, вакансии в IT службах:

In [17]:
# составим список слов, которые должны отсутствовать в названии вакансий

job_to_exclude = ['developer', 'development', 'programmatore', 'software', 'business process', 
                  'business-anal', 'business anal', 'funzionale', 'functional', 'governance', 'engineer',
                  'ingen', 'operational', 'operations', 'process', 'lab analyst', 'infrastructure', 
                  'science', 'scientist', 'financ', 'it solution', 'it strategy', 'support', 'it anal', 'esg']

In [18]:
# обновим датасет

parsed_df = (parsed_df.loc[
    parsed_df['job_title']
    .apply(lambda x: (sum([i in x.lower() for i in job_to_exclude]) == 0))]
            ).reset_index(drop=True)

#### Уровень вакансии

In [19]:
# посмотрим, какие уровни вакансии представлены в датасете

parsed_df['level'].unique()

array(['Entry level', None, 'Associate', 'Full-time', 'Internship',
       'Part-time'], dtype=object)

Уровни вакансии в целом соответствуют критериям поиска: представлены начальные позиции, включая стажировки.  
Уровень "Associate" может быть как начальным, так и middle в зависимости от грейдов в компании, далее проверим, не указан ли в этих случаях уровень в названии вакансии.  
Там, где уровнь вакансии получился "Full-time" и "Part-time", данных об требуемом уровне кандидата нет, и поэтому заменим такие значения на NaN:

In [20]:
parsed_df.loc[(parsed_df['level'] == 'Full-time') | (parsed_df['level'] == 'Part-time'), 'level'] = np.nan

In [21]:
# проверим, сколько в датасете вакансий без уровня кандидата

print('Число вакансий без указания уровня кандидата -', len(parsed_df.query('level.isna()')))

Число вакансий без указания уровня кандидата - 25


In [22]:
# посмотрим на названия вакансий, где не указан уровень кандидата

sorted(parsed_df.query('level.isna()')['job_title'].unique())

['Analista de datos BI',
 'Asset Data Analyst',
 'DATA ANALYST IT',
 'Data Analist - startersfunctie (Dutch speaking)',
 'Data Analyst',
 'Data Analyst (9 Months FTC)',
 'Data Analyst (Space & Planning)',
 'Data Analyst (m/w/d)',
 'Data Analyst - Hybrid',
 'Data Analyst - Hybrid Working',
 'Data Analyst - Logistics',
 'Data Analyst H/F',
 'Data analyst',
 'Datový analytik/vývojář']

Из вакансий выше только одна, связанная с анализом данных и BI, упоминает ожидаемый уровень кандидата - *'Data Analist - startersfunctie (Dutch speaking)'* (startersfunctie - начальная позиция). Проставим для неё начальный уровень:

In [23]:
parsed_df.loc[parsed_df['job_title'] == 'Data Analist - startersfunctie (Dutch speaking)',
              'level'] = 'Entry level'

In [24]:
# посмотрим на названия вакансий, где уровень - "Associate"

sorted(parsed_df.query('level == "Associate"')['job_title'].unique())

['(Junior) Consumer Business Intelligence Analyst (m/w/d)',
 '(Junior) Project Analyst (m/f/d)',
 'ANALISTA DE PROYECTO DIGITALIZACIÓN',
 'Analista Dati di Geodesia - CATEGORIA PROTETTA',
 'Analista Ecommerce, Web y App',
 'Analista de datos',
 'Analista de datos dept de elaborados (Guissona)',
 'Analista superior de datos',
 'Analityk',
 'Analityk danych',
 'Analyst',
 'Analyst - Marketing Effectiveness',
 'Analyst Sourcing & Planning',
 'Analytics & Reporting - Associate – Warsaw',
 'Analytics Consultant',
 'Analytics Consultant - Marketing Effectiveness',
 'Associate Database Analyst---Clinical Database Management',
 'BI Analyst',
 'BI Analyst (Pricing)',
 'BI Analyst (m/w/d)',
 'BI Analyst (m/w/d) Marketing',
 'BI Analyst, Power BI Champion, CEE based in Warsaw',
 'BI-Analyst (m/w/d)',
 'BUSINESS DATA ANALYSTE Senior H/F',
 'Business & Data Analyst',
 'Business Data Analyst (m/w/d)',
 'Business Data Analyst Lead\xa0H/F',
 'Business Intelligence Analyst',
 'Business Intelligence Ana

В названии некоторых вакансий указаны middle и senior уровни. Исключим такие вакансии:

In [25]:
# составим список уровней, которые не должны встречаться в названии вакансии

level_to_exclude = ['senior', 'superior', 'middle', 'mid', 'medior', 'lead', 'chef', 'principal',
                    'owner', 'head']

In [26]:
# исключим неподходящие вакансии из датасета

parsed_df = (parsed_df.loc[parsed_df['job_title']
                           .apply(lambda x: sum([i in x.lower() for i in level_to_exclude]) == 0)]
            ).reset_index(drop=True)

#### Регион

In [27]:
# проверим, что в датасет попали только страны Европы

sorted(parsed_df['country'].unique())

['Amsterdam Area',
 'Austria',
 'Belgium',
 'Berlin Metropolitan Area',
 'Brussels Metropolitan Area',
 'Bulgaria',
 'Cologne Bonn Region',
 'Czechia',
 'Denmark',
 'Eindhoven Area',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greater Banska Bystrica Area',
 'Greater Milan Metropolitan Area',
 'Greater Munster Area',
 'Greater Nuremberg Metropolitan Area',
 'Greater Palma de Mallorca Metropolitan Area',
 'Greater Paris Metropolitan Region',
 'Greater Pau Area',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Krakow Metropolitan Area',
 'Latvia',
 'Lisbon Metropolitan Area',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Monaco',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Prague Metropolitan Area',
 'Romania',
 'Rotterdam and The Hague',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'Warsaw Metropolitan Area',
 'Wroclaw Metropolitan Area']

Вакансии представлены только в Европе, однако в некоторых случаях указан лишь город/округ, и нет страны. Укажем для таких вакансий страну:

In [28]:
# составим словарь с названиями городов и соответствующих им стран

country_dict = {'Amsterdam Area': 'Netherlands', 'Athens Metropolitan Area':'Greece',
                'Berlin Metropolitan Area':'Germany', 'Brussels Metropolitan Area':'Belgium',
                'Cologne Bonn Region':'Germany', 'Copenhagen Metropolitan Area':'Denmark',
                'Eindhoven Area':'Netherlands', 'Greater Banska Bystrica Area':'Slovakia',
                'Greater Barcelona Metropolitan Area':'Spain',
                'Greater Dijon Area':'France', 'Greater Lyon Area':'France',
                'Greater Madrid Metropolitan Area':'Spain',
                'Greater Milan Metropolitan Area':'Italy',
                'Greater Munich Metropolitan Area':'Germany',
                'Greater Munster Area':'Ireland', 'Greater Nuremberg Metropolitan Area':'Germany',
                'Greater Oslo Region':'Norway', 'Greater Palma de Mallorca Metropolitan Area':'Spain',
                'Greater Paris Metropolitan Region':'France', 'Greater Pau Area':'France',
                'Greater Verona Metropolitan Area':'Italy',
                'Iasi Metropolitan Area':'Romania', 'Krakow Metropolitan Area':'Poland',
                'Lisbon Metropolitan Area':'Portugal', 'Prague Metropolitan Area':'Czechia',
                'Rotterdam and The Hague':'Netherlands', 'Stuttgart Region':'Germany',
                'Warsaw Metropolitan Area':'Poland', 'Wroclaw Metropolitan Area':'Poland'
               }

In [29]:
# добавим столбец с обновленными странами

parsed_df['country_new'] = parsed_df['country'].map(country_dict)
parsed_df.loc[parsed_df['country_new'].isna(), 'country_new'] = parsed_df['country']

---

In [30]:
# посмотрим на обновленный датасет

parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   job_title    526 non-null    object        
 1   company      526 non-null    object        
 2   city         526 non-null    object        
 3   country      526 non-null    object        
 4   level        502 non-null    object        
 5   workplace    487 non-null    object        
 6   posted_date  526 non-null    object        
 7   applicants   412 non-null    object        
 8   description  525 non-null    object        
 9   industry     495 non-null    object        
 10  size         495 non-null    object        
 11  date         526 non-null    datetime64[ns]
 12  country_new  526 non-null    object        
dtypes: datetime64[ns](1), object(12)
memory usage: 53.5+ KB


**Наблюдения:** Из 900 вакансий в датасете осталось 526 релевантных.

### Проверка на дубликаты - 2

Снова проверим датасет на наличие полных дубликатов: возможно, одна вакансия публиковалась несколько раз в один день.

In [31]:
# число полных дубликатов по итогам парсинга

sum(parsed_df.duplicated())

5

In [32]:
# удаляем полные дубликаты

parsed_df = parsed_df.drop_duplicates(ignore_index=True)

Проверим случаи, когда одна вакансия публиковалась в разные дни:

In [33]:
# число дублированных вакансий, опубликованных в разные дни

sum(parsed_df.duplicated(subset=['job_title','company','city','country','description']))

1

In [34]:
# удалим обнаруженные дубликаты

parsed_df = parsed_df.drop_duplicates(subset=['job_title','company','city','country','description'],
                                      ignore_index=True)

Проверим для вакансии без описания (индекс в датасете - 1), есть ли у нее дубликат:

In [35]:
# ищем дубликаты для вакансии без описания

parsed_df.query('(job_title == "Data Analyst - Logistics") and\
(company == "Resolute Recruitment") and (city == "Coventry")')

,job_title,company,city,country,level,workplace,posted_date,applicants,description,industry,size,date,country_new
1,Data Analyst - Logistics,Resolute Recruitment,Coventry,United Kingdom,None,On-site,1 week ago,None,NaN,NaN,NaN,2023-05-16,United Kingdom
2,Data Analyst - Logistics,Resolute Recruitment,Coventry,United Kingdom,None,On-site,1 week ago,None,"About the job\n \n\n \nData Analyst - Logistics ~ Permanent Role ~ Mon-Fri ~ Immediate Start ~Locations: Coventry, CV3Salary: 35,000 - 42,000 Per Year Shift Pattern: Monday to Friday 08:00 - 16:00*The correct candidate will also be able to work night shifts when required to observe this side of the business. Once the initial training period is complete you will be able to work from home as and when required. Job role: Analyse business data and provide detailed reports and reccomendations. Provide the business with regular reports to Quality Performance Supply key information to senior reviewers to assist decision making Ensuring actions are taken to speed up the issue resolution process Analyse general business processes to target cost saving & time saving efficiencies.. Report to business management and reccomend any changes. Monday to Friday Days, however the correct candidate must be prepared to work night shifts to look at this area of the business on occasion. What we are looking for: A proven background in data analytics & supplying detailed reports and reccomendations. Strong communication skills Flexibility to work nights on the rare occasion. Resolute Recruitment is acting as an Employment Agency in relation to this vacancy.Data Analysisdata, analysis, analytics, logistics, logistic, transport, business, profit, loss, performance, quality, management, coventry, warwickshire, nuneaton, rugby, leamington spa, warwick, stratford, wfh, work from home, analytical, Analyse, training, perm, permanent, jobs, job",NaN,NaN,2023-05-16,United Kingdom


Дубликат есть, удаляем строку без описания вакансии:

In [36]:
# удаляем вакансию без описания

parsed_df = parsed_df.dropna(subset=['description']).reset_index(drop=True)

Проверим, есть ли вакансии, у которых совпадает описание, компания и город:

In [37]:
# число вакансий с совпадающими описанием, компанией и городом

sum(parsed_df.duplicated(subset=['company','city','description']))

0

Посмотрим на вакансии, у которых совпадает название компании и город. Возможно, среди них есть неявные дубликаты: незначительно отличаются названия и детали в описании, но по сути это одна и та же вакансия. При поиске дубликатов мы исходим из предположения, что на одну должность может набираться несколько человек, и одной должности должна соответствовать одна вакансия.

In [38]:
# число вакансий с совпадающими компанией и городом

sum(parsed_df.duplicated(subset=['company','city']))

22

In [39]:
# составим список задублированных компаний и городов

duplicated_list = (parsed_df
                   .loc[parsed_df.duplicated(subset=['company','city']), ['company','city']]
                  ).reset_index()

In [40]:
# посмотрим на потенциально задублированные вакансии

(parsed_df
 .query('(company in @duplicated_list.company) and (city in @duplicated_list.city)')
 .sort_values(by=['company','city'])
)

job_title  \
99                                                             Data-analist   
122                                                            Data-analist   
265          Customer Insights Analyst (Bangkok Based, Relocation Provided)   
316                Statistical Analyst (Bangkok Based, Relocation Provided)   
101                                                         Analityk danych   
499                                                                Analityk   
8                                             Data Analyst - Hybrid Working   
12                                                    Data Analyst - Hybrid   
236                            Data Analyst - Alternance - Boursorama-(H/F)   
239                   Data Analyst Marketing Stratégique - Boursorama-(H/F)   
508                     Data Analyst / Risques de Crédit - Boursorama-(H/F)   
356                                                   Data Analyst - Milano   
470                                                    Stage | Data Analyst   
275                                                   Data Analyst - Milano   
468                                                    Stage | Data Analyst   
320                                                   Data Analyst - Milano   
477                                                    Stage | Data Analyst   
234                         Analista de datos dept de elaborados (Guissona)   
388                           Informático/a programador/a analista de datos   
39                                                         Data analyst H/F   
57                                                         Data Analyst H/F   
361                                       Analyst - Marketing Effectiveness   
416                          Analytics Consultant - Marketing Effectiveness   
210                                     Data Analyst (m/w/d) im Controlling   
370                       Analyst (m/f/d) Business Intelligence & Analytics   
159                 (Junior) Consumer Business Intelligence Analyst (m/w/d)   
181                 (Junior) Consumer Business Intelligence Analyst (m/w/d)   
93                       BI Analyst, Power BI Champion, CEE based in Warsaw   
478                                                      Commercial Analyst   
44                                                             Data Analyst   
50                                                             Data Analyst   
14                                                             Data Analyst   
22                                                          Data Analyst II   
256                                Business Intelligence Consultant (M/W/D)   
310                                                    Data Analyst (m/w/d)   
120                                                      BI-Analyst (m/w/d)   
167                                           Business Data Analyst (m/w/d)   
94                                                     Data Analyst Tableau   
164                                                    Data Analyst Costing   
287  Data Analyst in the area of IIoT / KI / Predictive Maintenance (m/f/d)   
440      Data Analyst im Bereich IIoT / KI / Predictive Maintenance (m/w/d)   
281  Data Analyst in the area of IIoT / KI / Predictive Maintenance (m/f/d)   
189                                                            Data Analyst   
258                                           Data Analyst Return Solutions   

                                      company                        city  \
99                         ABN AMRO Bank N.V.                  Amersfoort   
122                        ABN AMRO Bank N.V.                  Amersfoort   
265                                     Agoda                       Essen   
316                                     Agoda                       Essen   
101                                Billennium                      Poland   
499                                Billennium   

Неявные дубликаты есть: некоторые описания вакансий практически не отличаются, а названия совпадают либо переформулированы. В большинстве случаев совпадают также уровень и тип занятости. Иногда такие вакансии опубликованы в один день. Также есть задублированные вакансии, опубликованные с указанием разных городов, при этом в их описании упоминается, что место работы - офис в Милане.      

Проанализировав данные вакансии, мы решили исключить дубликаты со следующими индексами: 122, 265, 12, 275, 468, 320, 477, 57, 361, 181, 50, 14, 440.

In [41]:
# удалим неявные дубликаты

parsed_df = (parsed_df.drop(index=[122, 265, 12, 275, 468, 320, 477, 57, 361, 181, 50, 14, 440])
             .reset_index(drop=True))

In [42]:
# смотрим информацию о датасете без дубликатов

parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   job_title    506 non-null    object        
 1   company      506 non-null    object        
 2   city         506 non-null    object        
 3   country      506 non-null    object        
 4   level        485 non-null    object        
 5   workplace    469 non-null    object        
 6   posted_date  506 non-null    object        
 7   applicants   399 non-null    object        
 8   description  506 non-null    object        
 9   industry     478 non-null    object        
 10  size         478 non-null    object        
 11  date         506 non-null    datetime64[ns]
 12  country_new  506 non-null    object        
dtypes: datetime64[ns](1), object(12)
memory usage: 51.5+ KB


**Наблюдения:** После удаления дубликатов осталось 506 вакансий.

### Выделение hard skills

Проверим, какие из типичных хард скиллов указаны в требованиях к вакансии. Мы взяли список стандартных требований к хард-скиллам в дата-аналитике и дополнили его, просмотрев описание вакансий.  

Названия скиллов без упоминания конкретного иструмента (например, 'bi tools', 'bi', 'business intelligence', 'dashboard', 'dashboards', 'programming') укажем в конце списка, чтобы искать их в последнюю очередь, если не будут найдены конкретные инструменты.

In [43]:
# список hard skills

skills = ['datahub', 'api', 'github', 'google analytics', 'adobe analytics', 'ibm coremetrics', 'omniture',
          'gitlab', 'erwin', 'hadoop', 'spark', 'hive', 'databricks', 'aws', 'gcp', 'azure', 'excel',
          'redshift', 'bigquery', 'snowflake',  'hana', 'grafana', 'kantar', 'spss', 'a/b testing',
          'ab testing', 'asana', 'basecamp', 'jira', 'dbeaver','trello', 'miro', 'salesforce', 
          'rapidminer', 'thoughtspot',  'power point', 'docker', 'jenkins', 'integrate.io', 
          'talend', 'apache nifi', 'aws glue', 'pentaho', 'google data flow', 'azure data factory',
          'xplenty', 'skyvia', 'xtract.io', 'dataddo', 'ssis', 'hevo data',
          'informatica', 'oracle data integrator', 'k2view', 'cdata sync', 'querysurge', 'rivery', 
          'dbconvert', 'alooma', 'stitch', 'fivetran', 'matillion','streamsets', 'blendo', 
          'iri voracity', 'logstash', 'etleap', 'singer', 'apache camel', 'actian', 'airflow',
          'luidgi', 'datastage', 'python', 'vba', 'java script', 'julia', 'sql', 
          'matlab', 'java', 'html', 'c++', 'sas', 'tableau', 'looker', 'powerbi', 'cognos', 
          'microstrategy', 'spotfire', 'sap business objects', 'microsoft sql server',
          'oracle business intelligence', 'yellowfin', 'webfocus', 'sas visual analytics',
          'targit', 'izenda', 'sisense', 'statsbot', 'panorama', 'inetsoft', 'birst', 
          'domo', 'metabase', 'redash', 'power bi', 'alteryx', 'dataiku', 
          'microsoft office', 'sqlms', 'data models', 'data model', 'data modeling', 'data modelleren',
          'modèles statistiques', 'statistical analysis', 'google data studio', 'azure data studio',
          'relational db', 'power automate', 'nosql', 'sap crystal reports', 'postgresql', 'mysql',
          'microsoft powerapps', 'tibco spotfire', 'machine learning', 'mssql', 'oracle', 'ssrs', 'ssas',
          'power pivot', 'powerpivot', 'power query', 'powerquery', 'power automation',
          'dax', 'data factory', 'data lake', 'power fx', 'json', 'xml', 'terraform', 'notion',
          'ms office', 'office it', 'sharepoint', 'power-bi', 'js', 'r', 'mapr','sap',
          'powerpoint', 'mongodb', 'git', 'scrum', 'kanban', 'eviews', 'sequel', 'google cloud',
          'qliksense', 'qlik sense', 'qlikview',  'qlik view', 'qlik',
          'bi tools', 'bi', 'business intelligence', 'dashboard', 'dashboards', 'programming', 'coding',
          'english', 'inglese'
          ]
sorted(skills)

['a/b testing',
 'ab testing',
 'actian',
 'adobe analytics',
 'airflow',
 'alooma',
 'alteryx',
 'apache camel',
 'apache nifi',
 'api',
 'asana',
 'aws',
 'aws glue',
 'azure',
 'azure data factory',
 'azure data studio',
 'basecamp',
 'bi',
 'bi tools',
 'bigquery',
 'birst',
 'blendo',
 'business intelligence',
 'c++',
 'cdata sync',
 'coding',
 'cognos',
 'dashboard',
 'dashboards',
 'data factory',
 'data lake',
 'data model',
 'data modeling',
 'data modelleren',
 'data models',
 'databricks',
 'dataddo',
 'datahub',
 'dataiku',
 'datastage',
 'dax',
 'dbconvert',
 'dbeaver',
 'docker',
 'domo',
 'english',
 'erwin',
 'etleap',
 'eviews',
 'excel',
 'fivetran',
 'gcp',
 'git',
 'github',
 'gitlab',
 'google analytics',
 'google cloud',
 'google data flow',
 'google data studio',
 'grafana',
 'hadoop',
 'hana',
 'hevo data',
 'hive',
 'html',
 'ibm coremetrics',
 'inetsoft',
 'informatica',
 'inglese',
 'integrate.io',
 'iri voracity',
 'izenda',
 'java',
 'java script',
 'jenkin

In [44]:
# составим словарь унифицированных названий скиллов, чтобы избавиться от неявных дубликатов

skills_unified = {'azure':['azure','azure data factory','data factory','data lake','azure data studio'],
                  'power bi':['power bi','power-bi','powerbi'],
                  'data modeling':['data modeling','data model','data modelleren','data models',
                                   'modèles statistiques'],
                  'git':['github','gitlab'],
                  'ms office':['ms office','microsoft office','office it'],
                  'ms sql':['microsoft sql server','mssql','sqlms'],
                  'oracle':['oracle','oracle data integrator'],
                  'power automate':['power automate','power automation'],
                  'power point':['power point','powerpoint'],
                  'power pivot':['power pivot','powerpivot'],
                  'power query':['power query','powerquery'],
                  'qlik':['qlik','qlik sense','qliksense','qlik view','qlikview'],
                  'sap bi tools':['sap business objects','sap crystal reports'],
                  'ab testing':['ab testing','a/b testing'],
                  'aws':['aws','aws glue'],
                  'java script':['java script','js'],
                  'sas':['sas','sas visual analytics'],
                  'spotfire':['spotfire','tibco spotfire'],
                  'bi tools (unspecified)':['bi','bi tools','business intelligence','dashboard','dashboards'],
                  'programming language (unspecified)':['programming','coding'],
                  'english':['english','inglese']
                 }

# транспонируем словарь
skills_map = {}
for key in skills_unified.keys():
    for value in skills_unified[key]:
        skills_map.update({value:key})

In [45]:
# объявим функцию для выделения hard skills

def skills_list(description):
    # объявляем список, в который будем собирать скиллы для вакансии
    skills_list = []
    # идем по списку типичных хард скиллов и проверям наличие каждого элемента в описании вакансии
    for i in skills:      
        
        # задаем паттерн поиска
        # когда ищем в описании "sap", убеждаемся, что это не "sap business objects" и "sap crystal reports"
        if i == 'sap':
            skill = r'(\b|\W)' + re.escape(i) + (r'(?! c)' + '&' + r'(?! b)')
        # когда ищем в описании "sql", убеждаемся, что это не "microsoft sql server"
        elif i == 'sql':
            skill = r'(?!t)' + r'(\b|\W)' + re.escape(i) + r'(\b|\W)' + r'(?!s)'
        # в остальных случаях скилл должен быть отдельным словом
        # или, если в описании удалены пробелы (из-за чего не найдется скилл),
        # не требуем, чтобы скилл был отдельным словом, кроме некоторых случаев
        elif i not in ['api','aws','bi','git','hive','r','sas','ssis','ssas']:
            skill = r'(\b|\W)' + re.escape(i) + r'(\b|\W)' + '|' + re.escape(i.replace(' ',''))
        else:
            skill = r'(\b|\W)' + re.escape(i) + r'(\b|\W)'
                
        # находим скилл в описании
        if re.search(skill, description.lower()):
            
            # меняем название скилла на унифицированное 
            if i in skills_map:
                i = skills_map[i]
           
            # перед тем как добавить скилл 'bi tools (unspecified)' в список 'skills_list' проверяем,
            # что в этом списке нет конкретных BI инструментов
            if i == 'bi tools (unspecified)':
                check = 0
                for j in ['tableau','power bi','oracle business intelligence','birst','data studio',
                          'sap bi tools','inetsoft','izenda','looker','metabase','microstrategy',
                          'panorama','pentaho','redash','sas','sisense','spotfire','ssrs','targit',
                          'thoughtspot','yellowfin','qlick']:
                    if j in skills_list:
                        check += 1
                if (check == 0) and (i not in skills_list):
                    skills_list.append(i)
            
            # перед тем как добавить в список скилл 'programming language (unspecified)' проверяем,
            # что в списке нет конкретных языков программирования
            elif i == 'programming language (unspecified)':
                check = 0
                for j in ['python','r','eviews','java','java script','julia','matlab','vba','c++']:
                    if j in skills_list:
                        check += 1
                if (check == 0) and (i not in skills_list):
                    skills_list.append(i)
            
            # для других случаев просто добавляем в список скилл, если его еще там нет
            else:
                if i not in skills_list:
                    skills_list.append(i)
                
    # если вообще не нашли скиллы для вакансии, возвращаем nan, а непустой список сортируем по алфавиту
    if skills_list == []:
        skills_list = np.nan
    else:
        skills_list = sorted(skills_list)
    return skills_list

In [46]:
# создадим столбец с hard skills

parsed_df['skills'] = parsed_df['description'].apply(skills_list)

In [47]:
# убедимся, что скиллы распределены корректно

parsed_df[['description','skills']].head(50)

description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              About the job\n        \n\n \n                What You Will Achieve\nThis position will apply advanced manufacturing, science, and technology to support business and process improvements for the manufacture of small and / or large volume parenteral products. You will be a member of the Transformation and Strategy team. As a Data Analyst you will be responsible for mining, retrieving, organizing, and analyzing data to support the operations of a large manufacturing facility. Using the data, you will help to develop key performance indicators to demonstrate the effectiveness of processes and systems against business strategies.\nYour knowledge of manufacturing operations and computer systems/tools will make you a critical member of the team. Your strong business processes and workflow skills will help facilitate required gatherings for building and enhancing business process maps and strategies. Your innovative use of communication tools and techniques will facilitate in explaining difficult issues, establishing consensus between teams, and will create a collaborative teaming environment for your colleagues.\nMain Responsibilities\nInterpret data, analyze results using statistical techniques and provide ongoing reportsDevelop and implement databases, data collection systems, data analytics and other strategies that optimize statistical efficiency and qualityAcquire data from primary or secondary data sources and maintain databases/data systemsIdentify, analyze, and interpret trends or patterns in complex data sets Filter and “clean” data by reviewing reports, printouts, and performance indicatorsWork with management to prioritize business and information needs Identify and define new process improvement opportunities\nMust-Haves\nA Bachelor’s degree with at least three years of experience; OR a Master’s degree with more than one year of experience.Prior pharmaceutical and/or manufacturing experience requiredTechnical expertise regarding data models, database design development, data mining and segment

**Наблюдения:** Функция выделения скиллов работает, однако есть вакансии, где скиллы не были обнаружены. Посмотрим на них подробнее.

In [48]:
# посмотрим на количество вакансий без выделенных скиллов

len(parsed_df.query('skills.isna()'))

23

In [49]:
# посмотрим на описание вакансий без выделенных скиллов

parsed_df.query('skills.isna()')['description']

1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

**Наблюдения:** В описании данных вакансий действительно отсутствуют четкие технические требования. Таких вакансий немного относительно общего числа.

### Обработка признаков и категорий

Для построения дашборда выполним следующую предобработку:
- объединим компании, входящие в одну группу,
- унифицируем названия городов,
- выделим число откликов на вакансии в отдельную колонку,
- выделим хард скиллы в отдельный датасет.

Сферы деятельности и размер унифицированы и указываются компанией при создании аккаунта из списка, предоставляемого LinkedIn. Также унифицированы типы занятости, которые указываются при публикации вакансий. Следовательно, данные признаки не должны дублироваться и не требуют группировки/замены.

#### Тип занятости, сфера, размер

Убедимся, что тип занятости, сфера деятельности и размер компаний не дублируются:

In [50]:
# посмотрим на названия типов занятости

parsed_df['workplace'].unique()

array(['On-site', 'Hybrid', None, 'Remote'], dtype=object)

In [51]:
# посмотрим на названия сфер деятельности

sorted(parsed_df['industry'].astype(str).unique())

['Airlines/Aviation',
 'Apparel & Fashion',
 'Automotive',
 'Aviation & Aerospace',
 'Banking',
 'Biotechnology',
 'Building Materials',
 'Chemicals',
 'Computer Games',
 'Computer Software',
 'Consumer Electronics',
 'Consumer Goods',
 'Cosmetics',
 'Defense & Space',
 'Education Management',
 'Electrical & Electronic Manufacturing',
 'Entertainment',
 'Environmental Services',
 'Farming',
 'Financial Services',
 'Food & Beverages',
 'Food Production',
 'Gambling & Casinos',
 'Government Administration',
 'Higher Education',
 'Hospital & Health Care',
 'Hospitality',
 'Human Resources',
 'Individual & Family Services',
 'Information Services',
 'Information Technology & Services',
 'Insurance',
 'International Affairs',
 'International Trade & Development',
 'Internet',
 'Leisure, Travel & Tourism',
 'Luxury Goods & Jewelry',
 'Management Consulting',
 'Market Research',
 'Marketing & Advertising',
 'Mechanical Or Industrial Engineering',
 'Medical Device',
 'Mental Health Care',
 'Mi

In [52]:
# посмотрим на размер компаний

parsed_df['size'].unique()

array(['11-50 employees', nan, '10,001+ employees', '51-200 employees',
       '1,001-5,000 employees', '501-1,000 employees', '0-1 employees',
       '201-500 employees', '5,001-10,000 employees', '2-10 employees'],
      dtype=object)

**Наблюдения:** Типы занятости и размер компании не дублируются. В сферах деятельности возможно неявное дублирование (например, индустрии 'Airlines/Aviation' и 'Aviation & Aerospace', 'Consumer Goods' и 'Apparel & Fashion'/'Cosmetics'/'Food & Beverages'/'Food Production'). Чтобы понимать, насколько корректным было бы объединение сфер деятельности, нужно знать, чем занимается каждая компания в датасете, поэтому сферы деятельности оставим как есть.

#### Название компаний

In [53]:
# посмотрим на названия компаний

sorted(parsed_df['company'].unique())

['',
 '24S',
 'ABN AMRO Bank N.V.',
 'ACS Recruitment Solutions',
 'AGS S.\u200bp.\u200bA.\u200b',
 'AKQUINET',
 'ARI',
 'AUTODOC',
 'AXA Investment Managers',
 'Abbott',
 'Accenture Italia',
 'Action',
 'Adecco',
 'Agoda',
 'Airbus',
 'Albert Cliff',
 'Albéa Group',
 'Alteca',
 'Alvarez & Marsal',
 'Amadeus FiRe',
 'Antella Travel Recruitment',
 'Aon',
 'Applus+ Laboratories',
 'ArcelorMittal',
 'Artemys Agil-IT (groupe Artemys)',
 'AstraZeneca',
 'Ausy Belgium',
 'Autoriteit Financiële Markten',
 "Autostrade per l'Italia",
 'Avon',
 'Awin Global',
 'Axon Moore',
 'BCEE',
 'BINTER CANARIAS',
 'BNP Paribas',
 'Babcock',
 'Babcock International Group',
 'Bandai Namco Europe',
 'Banque Populaire Rives de Paris',
 'Behaviour – UK North',
 'Belfius',
 'Belgie Vacature Groep',
 'Benchmark International',
 'Biedronka',
 'BillEase',
 'Billennium',
 'Black Fox Solutions',
 'Blue Arrow',
 'Boursorama',
 'Bouygues Telecom',
 'Bridgestone EMIA',
 'BrightStone Group',
 "Burton's Biscuit Company",


In [54]:
# для одной вакансии не указано название компании: посмотрим, есть ли оно есть в описании

parsed_df.query('company == ""')

job_title company       city  country        level workplace  \
424  Data Analyst (m/w/d)          Osnabrück  Germany  Entry level    Hybrid   

    posted_date    applicants  \
424  1 week ago  2 applicants   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Название данной компании есть в описании - Metacrew service GmbH:

In [55]:
# укажем название в датасете

parsed_df.loc[parsed_df['company'] == '', 'company'] = 'Metacrew service GmbH'

Объединим следующих работодателей в группы компаний:

In [56]:
# составим словарь для группировки компаний
company_dict = {'AXA':['AXA','AXA Investment Managers'],
                'Babcock':['Babcock','Babcock International Group'],
                'Credit Agricole':['Credit Agricole Consumer Finance','Crédit Agricole Italia'],
                'Experis':['Experis','Experis Italia'],
                'NTT DATA':['NTT DATA Europe & Latam','NTT DATA Italia'],
                'PwC':['PwC España','PwC France','PwC Polska'],
                'Randstad':['Randstad Switzerland','Randstad Technologies Italia'],
                'SDG Group':['SDG Group España','SDG Group Italy'],
                'Santander':['Santander','Santander Bank Polska'],
                'Societe Generale':['Societe Generale',
                                    'Societe Generale Corporate and Investment Banking - SGCIB'],
                'TELUS International':['TELUS International','TELUS International AI Data Solutions']
               }

# транспонируем словарь
company_map = {}
for key in company_dict.keys():
    for value in company_dict[key]:
        company_map.update({value:key})

In [57]:
# объединяем работодателей в группы компаний

parsed_df['company_new'] = parsed_df['company'].map(company_map)
parsed_df.loc[parsed_df['company_new'].isna(), 'company_new'] = (parsed_df.loc[parsed_df['company_new'].isna(),
                                                                               'company'])

#### Названия городов

In [58]:
# проверим, что города в датасете называются одинаково

sorted(parsed_df['city'].unique())

['Ahlen',
 'Aix-en-Provence',
 'Alsónémedi',
 'Amersfoort',
 'Amstelveen',
 'Amsterdam',
 'Amsterdam Area',
 'Arconate',
 'Arluno',
 'Arnhem',
 'Athens',
 'Baierbrunn',
 'Barcelona',
 'Bari',
 'Basel',
 'Basingstoke',
 'Bath',
 'Belfast',
 'Bellinzago Lombardo',
 'Bergamo',
 'Bergen op Zoom',
 'Bergisch Gladbach',
 'Berlin',
 'Berlin Metropolitan Area',
 'Binasco',
 'Birkirkara',
 'Blackpool',
 'Boadilla del Monte',
 'Bodelshausen',
 'Bois-Colombes',
 'Bollate',
 'Bologna',
 'Bordeaux',
 'Boulogne-Billancourt',
 'Bracknell',
 'Bremen',
 'Brindisi',
 'Bristol',
 'Brussels',
 'Brussels Metropolitan Area',
 'Brussels Region',
 'Bucharest',
 'Buckinghamshire',
 'Budapest',
 'Bulgaria',
 'Calvignasco',
 'Cambridge',
 'Canegrate',
 'Carpiano',
 "Cassano d'Adda",
 "Cassina de' Pecchi",
 'Cerdanyola del Vallès',
 'Cernusco sul Naviglio',
 'Cesate',
 'Chappes',
 'Chaucer',
 'Chester',
 'Coimbra',
 'Cologne',
 'Cologne Bonn Region',
 'Colturano',
 'Copenhagen',
 'Cornaredo',
 'Corsico',
 'County

У некоторых городов есть приписки "Area", "Metropolitan Area", "Greater", "County" - уберем их. Иногда вместо города указана страна. В этом случае информации о городе нет, поэтому поставим пустые значения. Какие-то названия дублируются с немного разным написанием, исправим это.

In [59]:
# очистим названия от приписок типа "Area"

parsed_df['city_new'] = parsed_df['city']

for i in ['Metropolitan','Area','Greater','County','and The Hague']:
    parsed_df['city_new'] = parsed_df['city_new'].str.replace(i, '').str.strip()

In [60]:
# посмотрим на названия, содержащие слово "Region": возможно, какие-то из них - это города

sorted(parsed_df.query('city_new.str.contains("Region")')['city_new'].unique())

['Brussels Region', 'Cologne Bonn Region', 'Flemish Region', 'Paris  Region']

Регионы Кёльн - Бонн (Cologne Bonn Region) и Фламандский (Flemish Region) точными названиями городов заменить нельзя, поэтому поставим пустые значения. "Brussels Region" и "Paris  Region" можно заменить названиями соответствующих городов:

In [61]:
# ставим пропуски в названии города для регионов Кёльн - Бонн и Фламандского
parsed_df.loc[(parsed_df['city_new'] == 'Cologne Bonn Region') |\
              (parsed_df['city_new'] == 'Flemish Region'),
              'city_new'] = ''

# убираем "Region" из названий других городов
parsed_df['city_new'] = parsed_df['city_new'].str.replace('Region', '').str.strip()

In [62]:
# поставим пустые значения там, где вместо названия города - страна

parsed_df.loc[parsed_df['city_new'] == parsed_df['country_new'], 'city_new'] = ''

In [63]:
# составим словарь замен для названий городов

city_unified = {'Cracow':'Krakow',
                'Frankfurt am Main':'Frankfurt',
                'Košice':'Kosice',
                'Lisboa':'Lisbon',
                'Münster':'Munster',
                'Palma de Mallorca':'Palma',
                'Wrocław':'Wroclaw'
               }

In [64]:
# заменим дубликаты в названиях городов

for key, value in city_unified.items():
    parsed_df['city_new'] = parsed_df['city_new'].str.replace(key, value)

#### Число откликов

Создадим столбец с числом откликов на вакансию, без слова "applicants":

In [65]:
parsed_df['applicants_new'] = (parsed_df['applicants']
                               .apply(lambda x: x.split()[0] if x != None else x)
                              ).astype('Int64')

#### Hard skills

Соберем хард скиллы в отдельный датасет для последующей визуализации:

In [66]:
# составим датасет со скиллами для каждой вакансии

skills_df = parsed_df[['job_title','company_new','skills']].explode('skills').dropna().reset_index()
skills_df.columns = ['job_id','job_title','company','skills']
skills_df['job_id'] += 1

In [67]:
# посмотрим на первые 20 строк

skills_df.head(20)

,job_id,job_title,company,skills
0,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,data modeling
1,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,excel
2,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,sas
3,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,spss
4,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,sql
5,3,Data Analyst (Space & Planning),Mole Valley Farmers,excel
6,4,Data Analyst,FORFIRM,aws
7,4,Data Analyst,FORFIRM,data modeling
8,4,Data Analyst,FORFIRM,gcp
9,4,Data Analyst,FORFIRM,oracle


In [68]:
# посмотрим, по скольким вакансиям есть информация

skills_df['job_id'].nunique()

483

<a id='extraction'></a>
## Выгрузка данных

In [69]:
# сохраним в отдельный датасет информацию по вакансиям, ограничившись только необходимыми колонками

jobs_data = parsed_df[['job_title','company_new','industry','size','city_new','country_new','workplace',
                       'date','applicants_new']
                     ].reset_index()
jobs_data.columns = ['job_id','job_title','company','industry','size','city','country','workplace',
                     'date','applicants']
jobs_data['job_id'] += 1

In [70]:
# выгрузим датасет с вакансиями в формате csv

jobs_data.to_csv('linkedin_jobs.csv')

In [71]:
# выгрузим датасет со скиллами в формате csv

skills_df.to_csv('linkedin_skills.csv')

<a id='dash'></a>
## Дашборд

При составлении дашборда ориентировались на фильтры поиска вакансий в порядке их приоритета - страна и город, тип занятости, индустрия, размер компании. Данные признаки расположили слева (тк читаем слева направо) и в центре дашборда.

Ссылка на дашборд:

https://public.tableau.com/app/profile/mrmrzpn/viz/DALinkedInJobs/DataAnalystLinkedInJobs